<a href="https://colab.research.google.com/github/Monisha2604/Final_Year_Project/blob/master/Classification_ANN_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.context import SparkContext,SparkConf
from pyspark.sql import SparkSession
from pyspark import SQLContext

In [ ]:
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.types import DoubleType,IntegerType

In [ ]:
sc=SparkContext("local[2]","MultilayerPerceptron")
spark=SparkSession.builder.master("local[2]").appName("MultilayerPerceptron").getOrCreate()
sqlContext=SQLContext(sc)

In [ ]:
df=sqlContext.read.csv("/content/drive/MyDrive/MyTable_13_snigdha.csv",header=True,inferSchema=True)
print(df.printSchema())

root
 |-- objid: long (nullable = true)
 |-- ra: double (nullable = true)
 |-- dec: double (nullable = true)
 |-- psfMag_u: double (nullable = true)
 |-- psfMag_g: double (nullable = true)
 |-- psfMag_r: double (nullable = true)
 |-- psfMag_i: double (nullable = true)
 |-- psfMag_z: double (nullable = true)
 |-- dered_u: double (nullable = true)
 |-- dered_g: double (nullable = true)
 |-- dered_r: double (nullable = true)
 |-- dered_i: double (nullable = true)
 |-- dered_z: double (nullable = true)
 |-- fiberMag_u: double (nullable = true)
 |-- fiberMag_g: double (nullable = true)
 |-- fiberMag_r: double (nullable = true)
 |-- fiberMag_i: double (nullable = true)
 |-- fiberMag_z: double (nullable = true)
 |-- petroR50_r: double (nullable = true)
 |-- petroR50_z: double (nullable = true)
 |-- petroR90_r: double (nullable = true)
 |-- petroR90_z: double (nullable = true)
 |-- r: double (nullable = true)
 |-- i: double (nullable = true)
 |-- z: double (nullable = true)
 |-- redshift: doub

In [ ]:
df=df[df.psfMag_u!=-9999.0]

df=df[df.psfMag_g!=-9999.0]
df=df[df.psfMag_r!=-9999.0]
df=df[df.psfMag_i!=-9999.0]
df=df[df.psfMag_z!=-9999.0]
df=df[df.dered_u!=-9999.0]
df=df[df.dered_g!=-9999.0]
df=df[df.dered_r!=-9999.0]
df=df[df.dered_i!=-9999.0]
df=df[df.dered_z!=-9999.0]

df=df[df.fiberMag_u!=-9999.0]
df=df[df.fiberMag_g!=-9999.0]
df=df[df.fiberMag_r!=-9999.0]
df=df[df.fiberMag_i!=-9999.0]
df=df[df.fiberMag_z!=-9999.0]

df=df[df.petroR50_r!=-9999.0]
df=df[df.petroR50_z!=-9999.0]
df=df[df.petroR90_r!=-9999.0]
df=df[df.petroR90_z!=-9999.0]

df=df[df.r!=-9999.0]
df=df[df.i!=-9999.0]
df=df[df.z!=-9999.0]


In [ ]:
df.describe().show()

+-------+--------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+
|summary|               objid|                 ra|               dec|          psfMag_u|          psfMag_g|          psfMag_r|          psfMag_i|          psfMag_z|           dered_u|           dered_g|           dered_r|           dered_i|          dered_z|        fiberMag_u|        fiberMag_g|        fiberMag_r|        fiberMag_i|        fiberMag_z|        petroR50_r|        petroR50_z|       petroR90_r|        petroR90_z|                 r|                i|                 z|          re

In [ ]:
attributes=df.columns
#for i in range(len(attributes)):
#    print(attributes[i]," :",df.corr(attributes[i],"redshift"))

In [ ]:
df=df.drop("objid","ra","dec")
print(df.printSchema())

root
 |-- psfMag_u: double (nullable = true)
 |-- psfMag_g: double (nullable = true)
 |-- psfMag_r: double (nullable = true)
 |-- psfMag_i: double (nullable = true)
 |-- psfMag_z: double (nullable = true)
 |-- dered_u: double (nullable = true)
 |-- dered_g: double (nullable = true)
 |-- dered_r: double (nullable = true)
 |-- dered_i: double (nullable = true)
 |-- dered_z: double (nullable = true)
 |-- fiberMag_u: double (nullable = true)
 |-- fiberMag_g: double (nullable = true)
 |-- fiberMag_r: double (nullable = true)
 |-- fiberMag_i: double (nullable = true)
 |-- fiberMag_z: double (nullable = true)
 |-- petroR50_r: double (nullable = true)
 |-- petroR50_z: double (nullable = true)
 |-- petroR90_r: double (nullable = true)
 |-- petroR90_z: double (nullable = true)
 |-- r: double (nullable = true)
 |-- i: double (nullable = true)
 |-- z: double (nullable = true)
 |-- redshift: double (nullable = true)

None


In [ ]:
describe=df.describe().toPandas()
describe

,summary,psfMag_g,psfMag_r,psfMag_i,psfMag_z,dered_u,dered_g,dered_r,dered_i,dered_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,r,i,z,redshift
0,count,2429280,2429280,2429280,2429280,2429280,2429280,2429280,2429280,2429280,2429280,2429280,2429280,2429280,2429280,2429280,2429280,2429280,2429280
1,mean,21.466264234008563,20.463421008488332,19.883576505560058,19.490572486411523,21.984563734569313,20.553141212509566,19.506991573544486,18.93082651200435,18.59371937502935,22.65453073596289,21.527725587787412,20.487534032783614,19.89249716960207,19.51929580591793,19.580745583797224,18.985633644417724,18.634485815274736,0.6867449378489084
2,stddev,1.6583640742182884,1.423055409915492,1.2741065156414664,1.244813872814313,2.204401136207499,1.9901269483934518,1.8057082655894625,1.7190494334590851,1.7133292314898583,1.6117767634816462,1.653421140721453,1.462897146629538,1.344773516806486,1.3042327676613596,1.8012515997704157,1.7155376304086143,1.7107021498065549,0.7286797765964766
3,min,9.898472,9.470937,9.365116,9.529445,-13.59375,-7.685284,0.5439796,4.458445,6.233005,13.47047,14.59783,13.86967,13.81401,12.28998,9.474476,9.370869,9.673311,1.208629E-7
4,max,30.09237,26.50509,29.95165,24.39007,31.18573,31.53849,31.85576,30.85461,29.11098,29.18485,30.80903,27.73823,31.09943,24.9771,31.9436,30.88116,29.17408,7.05193


In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
def somefunc(value):
    if   value>0 and value<0.7: 
        return 0
    elif value>0.7 and value<1.41:
        return 1
    elif value>1.41 and value<2.11:
        return 2
    elif value>2.11 and value<2.82:
        return 3
    elif value>2.82 and value<3.52:
        return 4
    elif value>3.52 and value<4.23:
        return 5
    elif value>4.23 and value<5:
        return 6
    elif value>5 and value<5.6:
        return 7
    elif value>5.6 and value<6.3:
        return 8
    elif value>6.3 and value<7.05:
        return 9
    
#convert to a UDF Function by passing in the function and return type of function
udfsomefunc = F.udf(somefunc, IntegerType())
df= df.withColumn("label", udfsomefunc("redshift"))
df.show()

+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+----------+-----+
|psfMag_u|psfMag_g|psfMag_r|psfMag_i|psfMag_z| dered_u| dered_g| dered_r| dered_i| dered_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR50_z|petroR90_r|petroR90_z|       r|       i|       z|  redshift|label|
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+----------+-----+
|24.35021|22.78955|22.03197|21.07172|21.22284|24.85709|21.12559|20.30777| 19.2567|19.64961|  23.45947|  22.21595|  21.79943|  20.82268|  20.63929|  3.463256|  1.221438|  5.318553|  3.343638|20.38345|19.31294|19.69144| 0.8162744|    1|
|24.47392|22.43417|20.73903|19.74737|19.26174| 24.2802|21.56

In [ ]:
#import pyspark.sql.functions as F
#from pyspark.sql.types import *
#def somefunc(value):
#   if   value>0 and value<5: 
#        return 0
#    elif value>5 and value<7.051:
#        return 1
    
#convert to a UDF Function by passing in the function and return type of function
#udfsomefunc = F.udf(somefunc, IntegerType())
#df= df.withColumn("label", udfsomefunc("redshift"))
#df.show()

In [ ]:
df.printSchema()

root
 |-- psfMag_u: double (nullable = true)
 |-- psfMag_g: double (nullable = true)
 |-- psfMag_r: double (nullable = true)
 |-- psfMag_i: double (nullable = true)
 |-- psfMag_z: double (nullable = true)
 |-- dered_u: double (nullable = true)
 |-- dered_g: double (nullable = true)
 |-- dered_r: double (nullable = true)
 |-- dered_i: double (nullable = true)
 |-- dered_z: double (nullable = true)
 |-- fiberMag_u: double (nullable = true)
 |-- fiberMag_g: double (nullable = true)
 |-- fiberMag_r: double (nullable = true)
 |-- fiberMag_i: double (nullable = true)
 |-- fiberMag_z: double (nullable = true)
 |-- petroR50_r: double (nullable = true)
 |-- petroR50_z: double (nullable = true)
 |-- petroR90_r: double (nullable = true)
 |-- petroR90_z: double (nullable = true)
 |-- r: double (nullable = true)
 |-- i: double (nullable = true)
 |-- z: double (nullable = true)
 |-- redshift: double (nullable = true)
 |-- label: integer (nullable = true)



In [ ]:
assembler=VectorAssembler(
    inputCols=['psfMag_g',
               'psfMag_r',
               'psfMag_i',
               'psfMag_z',
               'dered_u', 
               'dered_g', 
               'dered_r', 
               'dered_i', 
               'dered_z', 
               'fiberMag_u', 
               'fiberMag_g', 
               'fiberMag_r', 
               'fiberMag_i', 
               'fiberMag_z', 
               'r', 
               'i', 
               'z'],
    outputCol="features"
)

In [ ]:
df=assembler.transform(df)

In [ ]:
df.printSchema()

root
 |-- psfMag_u: double (nullable = true)
 |-- psfMag_g: double (nullable = true)
 |-- psfMag_r: double (nullable = true)
 |-- psfMag_i: double (nullable = true)
 |-- psfMag_z: double (nullable = true)
 |-- dered_u: double (nullable = true)
 |-- dered_g: double (nullable = true)
 |-- dered_r: double (nullable = true)
 |-- dered_i: double (nullable = true)
 |-- dered_z: double (nullable = true)
 |-- fiberMag_u: double (nullable = true)
 |-- fiberMag_g: double (nullable = true)
 |-- fiberMag_r: double (nullable = true)
 |-- fiberMag_i: double (nullable = true)
 |-- fiberMag_z: double (nullable = true)
 |-- petroR50_r: double (nullable = true)
 |-- petroR50_z: double (nullable = true)
 |-- petroR90_r: double (nullable = true)
 |-- petroR90_z: double (nullable = true)
 |-- r: double (nullable = true)
 |-- i: double (nullable = true)
 |-- z: double (nullable = true)
 |-- redshift: double (nullable = true)
 |-- label: integer (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
label=round(df.groupBy().avg("label").collect()[0][0],2)
df=df.na.fill({"label":label})
df=df.na.drop()

In [ ]:
(trainingData,testData)=df.randomSplit([0.8,0.2])

In [ ]:
layers=[17,100,100,100,10]
mlp=MultilayerPerceptronClassifier(
    featuresCol="features",
    labelCol="label",
    layers=layers,
    maxIter=100,
    blockSize=128,
    seed=0)

In [ ]:
%%time
model=mlp.fit(trainingData)

CPU times: user 1min 4s, sys: 6.86 s, total: 1min 10s
Wall time: 5h 26min 21s


In [ ]:
result=model.transform(testData)

In [ ]:
result.show(10)

+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+----------+-----+--------------------+--------------------+--------------------+----------+
|psfMag_u|psfMag_g|psfMag_r|psfMag_i|psfMag_z| dered_u| dered_g| dered_r| dered_i| dered_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR50_z|petroR90_r|petroR90_z|       r|       i|       z|  redshift|label|            features|       rawPrediction|         probability|prediction|
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+----------+-----+--------------------+--------------------+--------------------+----------+
|15.51002|15.59608| 16.7501|16.71311|13.20851|  12.057|12.34764|13.36219|

In [ ]:
prediction_label = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
print("MLP test accuracy: " + str(evaluator.evaluate(prediction_label)))

MLP test accuracy: 0.7862831159290927


#1)[17,28,24,10]
iteration1 :
===>CPU times: user 317 ms, sys: 63.3 ms, total: 381 ms,Wall time: 35min 4s
===>accuracy:77%


#2)[17,30,40,10]
iteration2:
===>CPU times: user 518 ms, sys: 114 ms, total: 633 ms,Wall time: 55min 43s
===>accuracy:78.09%


#3)[17,50,50,10]
iteration3:
===>CPU times:CPU times: user 655 ms, sys: 142 ms, total: 797 ms,Wall time: 1h 16min 27s
===>accurarcy:76.17%

In [ ]:
redshift=np.array(df.select("reshift").collect())
import matplotlib.pyplot as plt
plt.hist(redshift,bins=10)

In [ ]:
redshift_class=np.array(df.select("label").collect())
import matplotlib.pyplot as plt
plt.hist(redshift_class,bins=10)